In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from collections import defaultdict
import ast


In [ ]:
def clique_percolation(G, k):
    """
    Implements the clique percolation method for community detection.
    k: int, clique size
    """
    # 1. Find all k-cliques
    cliques = list(nx.find_cliques(G))
    k_cliques = [c for c in cliques if len(c) >= k]

    # 2. Construct the clique graph
    clique_graph = nx.Graph()
    for i, c1 in enumerate(k_cliques):
        for j, c2 in enumerate(k_cliques[i+1:], i+1):
            if len(set(c1) & set(c2)) >= (k-1):  # If two cliques share k-1 nodes
                clique_graph.add_edge(i, j)

    # 3. Find connected components (these represent the communities)
    communities = []
    for component in nx.connected_components(clique_graph):
        community = set()
        for clique_id in component:
            community.update(k_cliques[clique_id])
        communities.append(community)

    return communities

In [3]:
def calculate_pairwise_accuracy(communities, G):
    total_pairs = 0
    correct_pairs = 0
    
    for community in communities:
        nodes = list(community)
        for i in range(len(nodes)):
            for j in range(i+1, len(nodes)):
                total_pairs += 1
                if G.nodes[nodes[i]]['in_playlist'] == G.nodes[nodes[j]]['in_playlist']:
                    correct_pairs += 1
    
    return correct_pairs / total_pairs if total_pairs > 0 else 0

In [4]:
def main():
    weeks = weeks = [
        '/Users/lunaliu/Desktop/Grad School/FA 2024/SI 608/week1.csv', 
        '/Users/lunaliu/Desktop/Grad School/FA 2024/SI 608/week2.csv', 
        '/Users/lunaliu/Desktop/Grad School/FA 2024/SI 608/week3.csv'
    ]
    results = []
    results = []

    for week_file in weeks:
        print(f"\nAnalyzing {week_file}")

        # load data
        df = pd.read_csv(week_file, index_col=0, low_memory=False)

        # load graph
        G = nx.Graph()
        for idx, row in df.iterrows():
            G.add_node(idx, in_playlist=row['in_playlist'])
            if isinstance(row['collaborators'], str):
                collaborators = ast.literal_eval(row['collaborators'])
                for collaborator in collaborators.keys():
                    if collaborator in df.index:
                        G.add_edge(idx, collaborator)

        print(f"The network has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")

        # Perform clique percolation for different values of k
        for k in [3, 4]:  # k=3 and k=4 are common choices
            print(f"\nPerforming clique percolation with k={k}")

            # Execute clique percolation
            communities = clique_percolation(G, k)

            if communities:
                # Calculate accuracy
                accuracy = calculate_pairwise_accuracy(communities, G)

                # Record results
                results.append({
                    'week': week_file,
                    'k': k,
                    'n_communities': len(communities),
                    'accuracy': accuracy,
                    'avg_community_size': np.mean([len(c) for c in communities])
                })

                print(f"Discovered {len(communities)} communities")
                print(f"Average community size: {results[-1]['avg_community_size']:.2f}")
                print(f"pairwaise accuracy: {accuracy:.3f}")

    # Save results
    results_df = pd.DataFrame(results)
    print("\nFinal results:")
    print(results_df)
    results_df.to_csv('community_percolation_results.csv')

In [5]:
if __name__ == "__main__":
    main()


Analyzing /Users/lunaliu/Desktop/Grad School/FA 2024/SI 608/week1.csv
The network has 4199 nodes and 19781 edges

Performing clique percolation with k=3
Discovered 191 communities
Average community size: 16.69
pairwaise accuracy: 0.896

Performing clique percolation with k=4
Discovered 115 communities
Average community size: 10.09
pairwaise accuracy: 0.972

Analyzing /Users/lunaliu/Desktop/Grad School/FA 2024/SI 608/week2.csv
The network has 4172 nodes and 19742 edges

Performing clique percolation with k=3
Discovered 187 communities
Average community size: 16.94
pairwaise accuracy: 0.896

Performing clique percolation with k=4
Discovered 113 communities
Average community size: 10.20
pairwaise accuracy: 0.972

Analyzing /Users/lunaliu/Desktop/Grad School/FA 2024/SI 608/week3.csv
The network has 4166 nodes and 19729 edges

Performing clique percolation with k=3
Discovered 185 communities
Average community size: 17.09
pairwaise accuracy: 0.898

Performing clique percolation with k=4
Dis